In [1]:
from models.tacotron2 import Tacotron2
import torch 
from torch import nn
from utils.utils import TacotronPreprocessor, TTSDataset
from utils.utils import reconstruct_audio
from torchaudio.functional import preemphasis
from torchaudio import transforms
import hyperparams as hps
import torchaudio


from torchaudio.utils import download_asset
SAMPLE_NOISE = download_asset("tutorial-assets/Lab41-SRI-VOiCES-rm1-babb-mc01-stu-clo-8000hz.wav")

In [2]:
from IPython import display

In [3]:
### Это запускать не надо в случае, если не модернизирвал датасет
temp = TTSDataset()
torch.save(temp, 'bin/TacotronDataset.pt')

In [4]:
dataset = torch.load('bin/TacotronDataset.pt')

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
vocab_size = dataset.preprocessor.vocab.shape[0] + 1
model = Tacotron2(vocab_size).to(DEVICE)
model.load_state_dict(torch.load('model_saves_clip\epoch_140_train_result'))

<All keys matched successfully>

In [6]:
def code_text(text):
    text = dataset.preprocessor.transform_single_text(text)
    text = torch.tensor(text).unsqueeze(0)
    return text

In [7]:
# ONLY FOR CUDA USAGE
def reconstruct_audio(mel, stops, noise = None):
    stops[-1] = 1
    stop = (stops==1).int().argmax()
    mel = torch.exp(mel[:, :stop])
    print('entered')
    inverse_melscale_transform = transforms.InverseMelScale(n_stft=hps.N_FFT // 2 + 1, n_mels = hps.N_MEL_FILTERBANKS, 
                                                            sample_rate=hps.SAMPLE_RATE, f_min=hps.FMIN, f_max=hps.FMAX, mel_scale='slaney').cuda()
    spectrogram = inverse_melscale_transform(mel.cuda())

    if noise is not None:
        spec_noise = inverse_melscale_transform(noise.cuda())

    print(spectrogram.min(), spec_noise.shape)
    spectrogram = torch.clip((spectrogram.transpose(0, 1) - 100*torch.mean(spec_noise.transpose(0, 1), dim=0)).transpose(0, 1), 0)

    print('inversed')
    transform = transforms.GriffinLim(n_fft=hps.N_FFT, win_length=hps.WINLEN, hop_length=hps.HOPLEN, n_iter=30).cuda()
    waveform = transform(spectrogram).detach().cpu()
    # waveform = preemphasis(waveform)
    print('transformed')
    return waveform


In [8]:
def return_mel(text = "Привет! Это проверка генерации речи из текста. Поздравляю!"):
    model.train(False)
    with torch.no_grad():
        sample_text = code_text(text).to(DEVICE)
        audio, stops, atention_context = model.predict(sample_text)
        stops = nn.functional.sigmoid(stops)[0]
        noise = audio[0][:, stops>0.5]
        audio = audio[0][:, stops<=0.5]
    return audio, stops, noise

In [9]:
audio, stops, noise = return_mel('Синтез речи это увлекательно. Я был прикольным ребенком, я не играл в погремушки, я мастерил шестеренки, метал снежки как из пушки. ')

In [10]:
temp = reconstruct_audio(audio, stops[stops<=0.5], noise)

entered
tensor(0., device='cuda:0') torch.Size([513, 904])
inversed
transformed


In [11]:
display.Audio(temp.numpy(), rate=22050)